In [163]:
import pandas as pd

In [164]:
def move_first_row_to_header(population_dataframe):
    df = population_dataframe.copy()
    new_header = df.iloc[0]
    df = df[1:]
    new_header[0] = "0"
    new_header.astype(int)
    new_header = [int(ele) for ele in new_header]
    new_header[0] = "County"
    df.columns = new_header
    return df

In [165]:
# This function only applies for data format of year 2000-2009
def structure_modification_for_old_xls(population_dataframe, state_name):

    df = population_dataframe.copy()
    # Drop unecessary columns and rows
    df.drop(df.columns[[1, 12, 13]], axis=1, inplace=True)
    df.drop(df.index[[0, 1, 3]], inplace=True)

    # Make row0 the header with a little modification tricks
    df = move_first_row_to_header(df)

    # Reset Index
    df.reset_index(inplace=True)
    df.drop("index", axis=1, inplace=True)

    # Some finetune
    df["County"] = df["County"].str[1:]
    df["State"] = state_name
    df = df.drop(df[df[2000].isnull()].index)

    return df

In [166]:
# This function only applies for data format of year 2010-2019
def structure_modification_for_new_xlsx(population_dataframe, state_name):

    df = population_dataframe.copy()

    # Drop unecessary columns and rows
    df.drop(df.columns[[1, 2]], axis=1, inplace=True)
    df.drop(df.index[[0, 1, 3]], inplace=True)

    # Make row0 the header with a little modification tricks
    df = move_first_row_to_header(df)

    # Reset Index
    df.reset_index(inplace=True)
    df.drop("index", axis=1, inplace=True)

    # Some finetune
    df["County"] = df["County"].str.split(", ").str[0]
    df["State"] = state_name
    df = df.drop(df[df[2010].isnull()].index)

    return df

In [167]:
# Read population data for all target states
state_list = ["FL", "TX", "WA", "NC", "SC", "PA"]

old_xls_list = []
new_xlsx_list = []
for state in state_list:
    # Read data from 2000 to 2009
    old_state_population = pd.read_excel(f"../00_source_data/Population/{state}_population_2000_2009.xls")
    old_state_population_modified = structure_modification_for_old_xls(old_state_population, state)
    old_xls_list.append(old_state_population_modified)
    # Read data from 2010 to 2019
    new_state_population = pd.read_excel(f"../00_source_data/Population/{state}_population_2010_2019.xlsx")
    new_state_population_modified = structure_modification_for_new_xlsx(new_state_population, state)
    new_state_population_modified["County"] = new_state_population_modified["County"].str[1:]
    new_xlsx_list.append(new_state_population_modified)


popuplation_2000 = pd.concat(old_xls_list)
popuplation_2010 = pd.concat(new_xlsx_list)
popuplation_2000.sample(10)


,County,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,State
56,Sullivan County,6577,6625.0,6592.0,6574.0,6554.0,6513.0,6487.0,6525.0,6462.0,6449.0,PA
47,Concho County,3963,3921.0,3959.0,3991.0,4018.0,4039.0,3957.0,4018.0,4108.0,4076.0,TX
117,Irion County,1758,1673.0,1697.0,1672.0,1629.0,1613.0,1649.0,1609.0,1571.0,1587.0,TX
62,Dickens County,2724,2653.0,2679.0,2670.0,2641.0,2563.0,2466.0,2461.0,2400.0,2423.0,TX
4,Ashe County,24480,24750.0,25029.0,25288.0,25684.0,25838.0,26134.0,26506.0,26858.0,27144.0,NC
152,Lynn County,6489,6444.0,6421.0,6232.0,6172.0,6178.0,6192.0,5989.0,5932.0,5927.0,TX
187,Potter County,113762,114307.0,115427.0,117067.0,117682.0,118771.0,119780.0,119537.0,119740.0,120118.0,TX
13,Bell County,239890,244299.0,249671.0,252922.0,258138.0,266017.0,273771.0,287752.0,297873.0,301050.0,TX
40,Saluda County,19191,19191.0,19180.0,19096.0,18904.0,19131.0,19310.0,19417.0,19503.0,19677.0,SC
94,Watauga County,42905,43853.0,44570.0,45095.0,45478.0,46390.0,47748.0,48765.0,50266.0,50706.0,NC


In [168]:
popuplation_2010.sample(10)

,County,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,State
31,Durham County,271357,276563.0,282659.0,288723.0,295137.0,300793.0,307826.0,312153.0,316979.0,321488.0,NC
109,Hockley County,22853,22931.0,23074.0,23408.0,23463.0,23310.0,23043.0,22977.0,22971.0,23021.0,TX
233,Van Zandt County,52579,52487.0,52207.0,52328.0,52781.0,53430.0,54367.0,55163.0,55966.0,56590.0,TX
158,Martin County,4809,4884.0,4980.0,5272.0,5462.0,5646.0,5616.0,5531.0,5681.0,5771.0,TX
128,Kaufman County,103872,105199.0,106553.0,108248.0,110872.0,114055.0,117904.0,122628.0,128279.0,136154.0,TX
14,Camden County,10009,10024.0,10003.0,10116.0,10278.0,10282.0,10397.0,10532.0,10676.0,10867.0,NC
246,Wilson County,43051,43667.0,44353.0,45217.0,46145.0,47178.0,48212.0,49211.0,50196.0,51070.0,TX
70,El Paso County,803576,819896.0,832310.0,830864.0,833783.0,831898.0,834949.0,837401.0,836825.0,839238.0,TX
45,Montgomery County,800874,805281.0,808672.0,812586.0,815753.0,817180.0,819791.0,824303.0,826924.0,830915.0,PA
249,Wood County,41977,42092.0,42383.0,42346.0,42764.0,43117.0,43753.0,44263.0,45157.0,45539.0,TX


In [169]:
# Data validity check
def validity_check(df):
    for column in df.columns:
        assert not df[column].isnull().any()

In [170]:
validity_check(popuplation_2000)
validity_check(popuplation_2010)

In [171]:
merged_population = popuplation_2000.merge(popuplation_2010, validate="1:1", indicator=True)
assert merged_population[merged_population["_merge"] != "both"].empty
merged_population.columns = merged_population.columns.astype(str)
merged_population.head()

,County,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,_merge
0,Alachua County,218611,221718.0,224614.0,227022.0,229867.0,233756.0,239506.0,242685.0,244888.0,...,249834.0,251520.0,252475.0,255456.0,259052.0,263959.0,266309.0,268851.0,269043.0,both
1,Baker County,22374,22620.0,23298.0,23555.0,24142.0,24832.0,25571.0,26212.0,26725.0,...,27049.0,27053.0,27009.0,27122.0,27355.0,27884.0,28254.0,28353.0,29210.0,both
2,Bay County,148393,150207.0,152741.0,155044.0,158804.0,162917.0,165644.0,165345.0,166267.0,...,169555.0,171757.0,174596.0,178289.0,181488.0,183634.0,184736.0,186240.0,174705.0,both
3,Bradford County,26064,26083.0,26306.0,27035.0,27703.0,28098.0,28506.0,28825.0,28961.0,...,28431.0,27051.0,26802.0,26552.0,26748.0,26740.0,27142.0,27752.0,28201.0,both
4,Brevard County,477819,486429.0,495425.0,504847.0,518722.0,529907.0,535138.0,539719.0,542378.0,...,544359.0,546966.0,550255.0,555548.0,565746.0,576874.0,587769.0,595203.0,601942.0,both


In [172]:
merged_population.to_parquet("../20_intermediate_files/Population_2000-2019.gzip", compression="gzip")
merged_population.to_csv("../20_intermediate_files/Population_2000-2019.csv")